In [644]:
import pandas as pd
import re
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import numpy as np

In [678]:
# bypass blocked webpage

import undetected_chromedriver as uc

options = webdriver.ChromeOptions() 
options.add_argument("start-maximized")
driver = uc.Chrome(options=options)

In [679]:
# get url of webpage

driver.get('https://www.therealreal.com/shop/women/clothing/designer_sweaters')

In [680]:
from selenium.webdriver.common.by import By

# empty dataframe
df = pd.DataFrame()

# add product name column
display_names = driver.find_elements(By.CLASS_NAME, "product-card__description")
display_name_list = [t.text for t in display_names][:6]
df['display_name'] = display_name_list

# add brand name column
brand = driver.find_elements(By.CLASS_NAME, "product-card__brand")
brand_list = [b.text for b in brand][:6]
df['brand_name'] = brand_list

# add product link column
lnks = driver.find_elements(By.XPATH, '//a[@data-source="plp"]')
prod_links = [lnks[i].get_attribute('href') for i in range(len(df))][:6]
df['product_url'] = prod_links

# add size column
elm = driver.find_elements(By.CLASS_NAME, 'product-card')

sizes = [re.findall(r'(?<=\nSize:\s\n-\n).*?(?=\n)', elm[i].text)[0] for i in range(len(df))][:6]
df['size'] = sizes

# add price column
prices = driver.find_elements(By.XPATH, '//div[@class="product-card__price" or @class="product-card__discount-price"]')
prices_list = [t.text for t in prices][:6]
df['price'] = [re.findall(r'\$[0-9]+', prices_list[i])[0] for i in range(len(df))]

df


,display_name,brand_name,product_url,size,price
0,Striped Crew Neck Sweater,JOHN VARVATOS,https://www.therealreal.com/products/women/clo...,M,$70
1,Linen V-Neck Sweater,BRUNELLO CUCINELLI,https://www.therealreal.com/products/women/clo...,XXS,$345
2,Sweater,LA DETRESSE,https://www.therealreal.com/products/women/clo...,XS,$22
3,2010 V-Neck Sweater,OSCAR DE LA RENTA,https://www.therealreal.com/products/women/clo...,XL,$125
4,Crew Neck Sweater,ESCADA,https://www.therealreal.com/products/women/clo...,M,$90
5,Scoop Neck Sweater,VINCE,https://www.therealreal.com/products/women/clo...,M,$80


In [684]:
driver.get('https://www.therealreal.com/shop/women/clothing/designer_sweaters')

In [685]:
description = []
colors = []
product_material = []
conditions = []

for i in range(1,7):

    # click each product to get more details
    li = driver.find_elements(By.XPATH, '//a[@data-position="{}"]'.format(i))[0]
    li.click()

    # product description
    desc = driver.find_elements(By.XPATH, '//div[@class="description"]')[0].text

    # product color
    color = re.findall(r'(?<=\n).*?(?=\n)', desc)[0]

    # product material
    fabric = driver.find_elements(By.XPATH, '//ul[@class="product-details-group"]')[0].text
    prod_material = re.findall(r'(?<=Fabric:\s).*', fabric)[0]

    # product condition
    condition = driver.find_elements(By.XPATH, '//li[@class="condition__attribute--active condition__attribute"]')[0].text

    driver.get('https://www.therealreal.com/shop/women/clothing/designer_sweaters')

    description.append(desc)
    colors.append(color)
    product_material.append(prod_material)
    conditions.append(condition)

In [686]:
driver.get('https://www.therealreal.com/shop/women/clothing/designer_sweaters')

In [687]:
# add lists extracted from above cell to df

df['description'] = [re.sub('\n', '. ', description[i]) for i in range(len(description))]
df['color'] = colors
df['product_material'] = product_material
df['low_level'] = [df['display_name'][i].rsplit(None, 1)[-1] for i in range(len(df))]
page_title = driver.find_elements(By.XPATH, '//h1[@class="plp-cms__content-title"]')[0].text
df['gender'] = page_title.rsplit()[0]
df['condition'] = conditions
df['secondhand'] = np.where(df['condition'] !='Pristine', True, False)
df

,display_name,brand_name,product_url,size,price,description,color,product_material,low_level,gender,condition,secondhand
0,Striped Crew Neck Sweater,JOHN VARVATOS,https://www.therealreal.com/products/women/clo...,M,$70,John Varvatos Pullover. Black. Striped. Long S...,Black,"43% Nylon, 35% Kid Mohair, 22% Wool",Sweater,WOMEN'S,Excellent,True
1,Linen V-Neck Sweater,BRUNELLO CUCINELLI,https://www.therealreal.com/products/women/clo...,XXS,$345,Brunello Cucinelli Linen Cardigan. Blue. Sequi...,Blue,"75% Linen, 25% Silk; Lining 100% Polyamide",Sweater,WOMEN'S,Very Good,True
2,Sweater,LA DETRESSE,https://www.therealreal.com/products/women/clo...,XS,$22,La Detresse Sweater. Blue. Fit:Knitwear by La ...,Blue,"80% Cotton, 20% Polyester",Sweater,WOMEN'S,Pristine,False
3,2010 V-Neck Sweater,OSCAR DE LA RENTA,https://www.therealreal.com/products/women/clo...,XL,$125,Oscar de la Renta Cropped Sweater. From the Re...,From the Resort 2010 Collection,100% Cotton,Sweater,WOMEN'S,Very Good,True
4,Crew Neck Sweater,ESCADA,https://www.therealreal.com/products/women/clo...,M,$90,Escada Pullover. Black. Short Sleeve with Crew...,Black,"77% Viscose, 23% Polyamide",Sweater,WOMEN'S,Excellent,True
5,Scoop Neck Sweater,VINCE,https://www.therealreal.com/products/women/clo...,M,$80,Vince Pullover. Neutrals. Long Sleeve with Sco...,Neutrals,100% Wool,Sweater,WOMEN'S,Excellent,True


In [747]:
# extract image links

imgs = []
img_lnks = []

for i in range(1, len(df)+1):
    img_lnk = driver.find_elements(By.XPATH, '/html/body/div[3]/div[1]/main/div[2]/div[4]/div[6]/div/div[{}]/a/div/div[1]/div[1]/img[1]'.format(i))[0]
    img_lnks.append(img_lnk.get_attribute('src'))

img_lnks

['https://product-images.therealreal.com/JVA47087_1_enlarged.jpg',
 'https://product-images.therealreal.com/BRU211119_1_enlarged.jpg',
 'https://product-images.therealreal.com/WLDRS20071_1_enlarged.jpg',
 'https://product-images.therealreal.com/OSC178824_1_enlarged.jpg',
 'data:image/svg+xml;charset=utf-8,%3Csvg%20xmlns%3D%27http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%27%20width%3D%27220%27%20height%3D%27290%27%3E%3Crect%20width%3D%27100%25%27%20height%3D%27100%25%27%20style%3D%27fill%3A%20white%27%2F%3E%3C%2Fsvg%3E',
 'data:image/svg+xml;charset=utf-8,%3Csvg%20xmlns%3D%27http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%27%20width%3D%27220%27%20height%3D%27290%27%3E%3Crect%20width%3D%27100%25%27%20height%3D%27100%25%27%20style%3D%27fill%3A%20white%27%2F%3E%3C%2Fsvg%3E']

In [751]:
# rearrange column names

cols = ['display_name', 'product_material', 'color', 'size', 'price', 'product_url', 'brand_name', 'description', 'low_level', 'gender', 'secondhand', 'condition']

final_df = df[cols]
final_df['image_links'] = img_lnks
final_df

/var/folders/vb/bprs19k12d92r7xpmxnv1rx40000gn/T/ipykernel_9347/3853933782.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['image_links'] = img_lnks


,display_name,product_material,color,size,price,product_url,brand_name,description,low_level,gender,secondhand,condition,image_links
0,Striped Crew Neck Sweater,"43% Nylon, 35% Kid Mohair, 22% Wool",Black,M,$70,https://www.therealreal.com/products/women/clo...,JOHN VARVATOS,John Varvatos Pullover. Black. Striped. Long S...,Sweater,WOMEN'S,True,Excellent,https://product-images.therealreal.com/JVA4708...
1,Linen V-Neck Sweater,"75% Linen, 25% Silk; Lining 100% Polyamide",Blue,XXS,$345,https://www.therealreal.com/products/women/clo...,BRUNELLO CUCINELLI,Brunello Cucinelli Linen Cardigan. Blue. Sequi...,Sweater,WOMEN'S,True,Very Good,https://product-images.therealreal.com/BRU2111...
2,Sweater,"80% Cotton, 20% Polyester",Blue,XS,$22,https://www.therealreal.com/products/women/clo...,LA DETRESSE,La Detresse Sweater. Blue. Fit:Knitwear by La ...,Sweater,WOMEN'S,False,Pristine,https://product-images.therealreal.com/WLDRS20...
3,2010 V-Neck Sweater,100% Cotton,From the Resort 2010 Collection,XL,$125,https://www.therealreal.com/products/women/clo...,OSCAR DE LA RENTA,Oscar de la Renta Cropped Sweater. From the Re...,Sweater,WOMEN'S,True,Very Good,https://product-images.therealreal.com/OSC1788...
4,Crew Neck Sweater,"77% Viscose, 23% Polyamide",Black,M,$90,https://www.therealreal.com/products/women/clo...,ESCADA,Escada Pullover. Black. Short Sleeve with Crew...,Sweater,WOMEN'S,True,Excellent,"data:image/svg+xml;charset=utf-8,%3Csvg%20xmln..."
5,Scoop Neck Sweater,100% Wool,Neutrals,M,$80,https://www.therealreal.com/products/women/clo...,VINCE,Vince Pullover. Neutrals. Long Sleeve with Sco...,Sweater,WOMEN'S,True,Excellent,"data:image/svg+xml;charset=utf-8,%3Csvg%20xmln..."


In [766]:
import sql 
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine

In [770]:
engine = create_engine('sqlite://', echo = False)
final_df.to_sql('real_real', con = engine)

engine.execute("SELECT * FROM real_real").fetchall()


[(0, 'Striped Crew Neck Sweater', '43% Nylon, 35% Kid Mohair, 22% Wool', 'Black', 'M', '$70', 'https://www.therealreal.com/products/women/clothing/knitwear/john-varvatos-striped-crew-neck-sweater-f1g51', 'JOHN VARVATOS', 'John Varvatos Pullover. Black. Striped. Long Sleeve with Crew Neck', 'Sweater', "WOMEN'S", 1, 'Excellent', 'https://product-images.therealreal.com/JVA47087_1_enlarged.jpg'),
 (1, 'Linen V-Neck Sweater', '75% Linen, 25% Silk; Lining 100% Polyamide', 'Blue', 'XXS', '$345', 'https://www.therealreal.com/products/women/clothing/knitwear/brunello-cucinelli-linen-v-neck-sweater-f59k2', 'BRUNELLO CUCINELLI', 'Brunello Cucinelli Linen Cardigan. Blue. Sequin Embellishments. Three-Quarter Sleeve with V-Neck. Button Closure at Front. Designer Fit: Sweaters by Brunello Cucinelli typically fit true to size.', 'Sweater', "WOMEN'S", 1, 'Very Good', 'https://product-images.therealreal.com/BRU211119_1_enlarged.jpg'),
 (2, 'Sweater', '80% Cotton, 20% Polyester', 'Blue', 'XS', '$22', 'ht